# Modelling for predicting the spring onset of natural vegetation across the Northern Hemisphere

https://doi.org/10.1016/j.ecolind.2021.108126


## Data

- Observation data from the USA National Phenology Network (USA-NPN)
- MODIS land cover type product and the land cover dynamics product
- Daymet gridded climate data covering Northern America
- Climate data obtained from Global Land Data Assimilation System (GLDAS) covering the Northern Hemisphere

### USA-NPN

- Observation records of vegetation phenology from 2895 sites from 2007 to 2017
- Four vegetation types defined in the USA-NPN dataset, including deciduous broadleaf forests (DBF), evergreen broadleaf forests (EBF), graminoid (GRA), and drought deciduous broadleaf forests (DDBF)
- Spring onset: breaking leaf buds for DBF, breaking leaf buds for ENF, breaking leaf buds for DDBF, and initial growth of grasses/sedges for GRA
- Averaging the timing of spring onsets for all records at the same site in the same year
- Excluding observation data in the sites located in the pixels that are classified as urban and build-up in the MODIS land cover maps

In [5]:
import springtime
from springtime.datasets.NPNPhenor import NPNPhenor, npn_species, npn_phenophases

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import math
import folium
import os
import glob
import tempfile

In [7]:
current_path = os.getcwd()
current_path

'/home/jovyan/springtime/docs/notebooks'

In [8]:
# List IDs and names for available species, phenophases
species = npn_species()
species

species_id,common_name,genus,...,family_id,family_name,family_common_name
120,"""'ohi'a l...",'Metrosid...,...,301,'Myrtaceae','Myrtle F...
1436,'absinthi...,'Artemisia',,242,'Asterace...,'Aster Fa...
1227,'Acadian ...,'Empidonax',,154,'Tyrannid...,'Tyrant F...
1229,'acorn wo...,'Melanerp...,,158,'Picidae','Woodpeck...
...,...,...,,...,...,...
228,'Yoshino ...,'Prunus',,325,'Rosaceae','Rose Fam...
1043,'youth on...,'Tolmiea',,332,'Saxifrag...,'Saxifrag...
1395,'zebra-ta...,'Callisau...,,209,'Phrynoso...,'Zebra-ta...
2188,'zigzag s...,'Tradesca...,,260,'Commelin...,'Spiderwo...


In [9]:
print(species.head(1))
# It's not easy to check the column names. I printed the first row and copy-pasted the names here:
# species_id, common_name, genus genus_id, genus_common_name, species, kingdom, itis_taxonomic_sn,functional_type, class_id, class_common_name, 
# class_name, order_id, order_common_name, order_name family_id, family_name, family_common_nam

  species_id  common_name        genus genus_id     genus_common_name
1        120 'ohi'a lehua Metrosideros      798 Lehuas (Metrosideros)
     species kingdom itis_taxonomic_sn     functional_type class_id
1 polymorpha Plantae             27259 Evergreen broadleaf       15
  class_common_name    class_name order_id                    order_common_name
1  Flowering Plants Magnoliopsida       89 Myrtle and Evening-primrose Families
  order_name family_id family_name family_common_name
1   Myrtales       301   Myrtaceae      Myrtle Family



In [10]:
#help(species)
species.to_csvfile(current_path+"/species.csv")
species_df = pd.read_csv(current_path+"/species.csv")
species_df

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
1,120,'ohi'a lehua,Metrosideros,798,Lehuas (Metrosideros),polymorpha,Plantae,27259.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,89,Myrtle and Evening-primrose Families,Myrtales,301,Myrtaceae,Myrtle Family
2,1436,absinthium,Artemisia,437,Sagebrushes (Artemisia),absinthium,Plantae,35445.0,Forb,15,Flowering Plants,Magnoliopsida,69,"Aster, Bellflower and Buckbean Families",Asterales,242,Asteraceae,Aster Family
3,1227,Acadian flycatcher,Empidonax,612,Empidonax Flycatchers (Empidonax),virescens,Animalia,178339.0,Bird,5,Birds,Aves,31,Perching Birds,Passeriformes,154,Tyrannidae,Tyrant Flycatchers
4,1229,acorn woodpecker,Melanerpes,790,Melanerpine Woodpeckers (Melanerpes),formicivorus,Animalia,178189.0,Bird,5,Birds,Aves,33,Woodpeckers,Piciformes,158,Picidae,Woodpeckers
5,2110,Adam and Eve,Aplectrum,1285,Adam and Eves (Aplectrum),hyemale,Plantae,43489.0,Forb,15,Flowering Plants,Magnoliopsida,68,"Asparagas, Iris, Orchid and Aloe Families",Asparagales,307,Orchidaceae,Orchid Family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,1671,yerba mansa,Anemopsis,413,Yerba Mansa (Anemopsis),californica,Plantae,18223.0,Forb,15,Flowering Plants,Magnoliopsida,92,Birthwort and Lizard's-tail Families,Piperales,331,Saururaceae,Lizard's-tail Family
1752,228,Yoshino cherry,Prunus,933,Cherries (Prunus),yedoensis,Plantae,836663.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,325,Rosaceae,Rose Family
1753,1043,youth on age,Tolmiea,1058,Youth on Ages (Tolmiea),menziesii,Plantae,24533.0,Forb,15,Flowering Plants,Magnoliopsida,98,"Currant, Witch-hazel and Saxifrage Families",Saxifragales,332,Saxifragaceae,Saxifrage Family
1754,1395,zebra-tailed lizard,Callisaurus,479,Zebra-tailed Lizards (Callisaurus),draconoides,Animalia,173906.0,Reptile,10,Reptiles,Reptilia,54,Snakes and Lizards,Squamata,209,Phrynosomatidae,Zebra-tailed and Horned Lizards


In [11]:
species_df_D = species_df.loc[species_df['functional_type'] == "Deciduous broadleaf"]
species_df_D

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
11,1174,Allegheny serviceberry,Amelanchier,402,Serviceberries (Amelanchier),laevis,Plantae,182046.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,325,Rosaceae,Rose Family
24,1446,alternateleaf dogwood,Cornus,560,Dogwoods (Cornus),alternifolia,Plantae,27813.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,75,"Dogwood, Hydrangea and Tupelo Families",Cornales,262,Cornaceae,Dogwood Family
27,93,American basswood,Tilia,1056,Lindens (Tilia),americana,Plantae,21536.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,88,Mallow and Mezereum Families,Malvales,294,Malvaceae,Mallow Family
28,812,American beautyberry,Callicarpa,477,Beautyberries (Callicarpa),americana,Plantae,32144.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,83,"Mint, Olive and Plantain Families",Lamiales,286,Lamiaceae,Mint Family
29,79,American beech,Fagus,645,Beeches (Fagus),grandifolia,Plantae,19462.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,79,"Birch, Beech, Walnut and Bayberry Families",Fagales,271,Fagaceae,Beech Family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713,217,Woods' rose,Rosa,965,Roses (Rosa),woodsii,Plantae,24847.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,325,Rosaceae,Rose Family
1717,1905,woolly pawpaw,Asimina,442,Pawpaws (Asimina),incana,Plantae,18109.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,86,Custard-apple and Magnolia Families,Magnoliales,233,Annonaceae,Custard-apple Family
1726,97,yellow birch,Betula,457,Birches (Betula),alleghaniensis,Plantae,19481.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,79,"Birch, Beech, Walnut and Bayberry Families",Fagales,245,Betulaceae,Birch Family
1729,1172,yellow buckeye,Aesculus,383,Buckeyes (Aesculus),flava,Plantae,837871.0,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,97,"Sumac, Rue and Soapberry Families",Sapindales,329,Sapindaceae,Soapberry Family


In [17]:
species_df_E = species_df.loc[species_df['functional_type'] == "Evergreen broadleaf"]
species_df_E

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
1,120,'ohi'a lehua,Metrosideros,798,Lehuas (Metrosideros),polymorpha,Plantae,27259.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,89,Myrtle and Evening-primrose Families,Myrtales,301,Myrtaceae,Myrtle Family
14,1861,alpine azalea,Kalmia,731,Laurels (Kalmia),procumbens,Plantae,565766.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,77,"Heath, Phlox and Pitcher-plant Families",Ericales,268,Ericaceae,Heath Family
36,1717,American century plant,Agave,385,Agaves (Agave),americana,Plantae,182660.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,68,"Asparagas, Iris, Orchid and Aloe Families",Asparagales,241,Asparagaceae,Asparagas Family
48,1982,American holly,Ilex,719,Hollies (Ilex),opaca,Plantae,27982.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,66,Holly Family,Aquifoliales,236,Aquifoliaceae,Holly Family
117,1068,banana yucca,Yucca,1103,Yuccas (Yucca),baccata,Plantae,43134.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,68,"Asparagas, Iris, Orchid and Aloe Families",Asparagales,241,Asparagaceae,Asparagas Family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,766,whitethorn ceanothus,Ceanothus,506,Ceanothuses (Ceanothus),cordulatus,Plantae,28458.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,323,Rhamnaceae,Buckthorn Family
1679,222,wild banyantree,Ficus,650,Figs (Ficus),citrifolia,Plantae,19094.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,299,Moraceae,Mulberry Family
1721,1351,Wright's silktassel,Garrya,664,Silktassels (Garrya),wrightii,Plantae,27839.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,80,Silk-tassel Family,Garryales,273,Garryaceae,Silk-tassel Family
1723,1467,yaupon,Ilex,719,Hollies (Ilex),vomitoria,Plantae,27986.0,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,66,Holly Family,Aquifoliales,236,Aquifoliaceae,Holly Family


In [12]:
ls_sp1 = np.sort(list(species_df_D["species_id"]))
ls_sp1

array([   1,    2,    3,    4,    7,   12,   20,   27,   28,   29,   33,
         35,   36,   59,   60,   61,   62,   63,   64,   65,   67,   68,
         69,   70,   71,   72,   73,   74,   75,   76,   77,   79,   80,
         81,   82,   83,   86,   87,   88,   89,   90,   91,   92,   93,
         94,   95,   96,   97,   98,   99,  100,  101,  102,  201,  206,
        211,  214,  216,  217,  227,  228,  292,  293,  303,  309,  312,
        314,  315,  316,  319,  320,  321,  322,  325,  326,  328,  430,
        444,  704,  713,  717,  719,  723,  724,  764,  765,  770,  777,
        778,  779,  780,  781,  783,  784,  789,  799,  802,  804,  810,
        812,  823,  824,  827,  829,  835,  836,  839,  844,  871,  872,
        873,  884,  885,  897,  915,  919,  920,  923,  935,  940,  941,
        942,  944,  949,  952,  962,  970,  976,  977,  982,  983,  988,
        989,  992,  993,  994,  996,  997,  999, 1000, 1001, 1006, 1007,
       1008, 1009, 1010, 1015, 1019, 1023, 1030, 10

In [18]:
ls_sp2 = np.sort(list(species_df_E["species_id"]))
ls_sp2

array([ 103,  104,  106,  107,  109,  110,  111,  112,  113,  114,  115,
        116,  117,  119,  120,  121,  127,  218,  221,  222,  226,  298,
        306,  311,  313,  318,  324,  327,  329,  435,  441,  442,  443,
        701,  702,  705,  706,  707,  716,  721,  722,  743,  760,  761,
        766,  769,  785,  790,  793,  830,  857,  860,  861,  877,  878,
        879,  880,  887,  895,  903,  904,  905,  907,  909,  924,  925,
        933,  979,  980,  981,  985,  995, 1003, 1011, 1055, 1057, 1058,
       1067, 1068, 1069, 1160, 1166, 1170, 1178, 1322, 1323, 1324, 1346,
       1349, 1351, 1359, 1360, 1368, 1430, 1443, 1444, 1467, 1479, 1490,
       1593, 1594, 1602, 1609, 1626, 1628, 1644, 1681, 1689, 1717, 1720,
       1766, 1799, 1824, 1845, 1846, 1861, 1870, 1887, 1892, 1910, 1922,
       1927, 1934, 1938, 1961, 1968, 1969, 1970, 1975, 1976, 1982, 1985,
       2006, 2054, 2057, 2064, 2139])

In [13]:
phenophases = npn_phenophases()
help(phenophases)
# phenophases.to_csvfile(current_path+"/phenophases.csv")
# phenophases_df = pd.read_csv(current_path+"/phenophases.csv")
# phenophases_df

    phenophase_id                  phenophase_name phenophase_category  color
1              56                      First leaf               Leaves   <NA>
2              57             75% leaf elongation               Leaves   <NA>
3              58                    First flower              Flowers   <NA>
4              59                      Last flower             Flowers   <NA>
5              60                First fruit ripe               Fruits   <NA>
6              61            50% of leaves colored              Leaves   <NA>
7              63             50% of leaves fallen              Leaves   <NA>
8              71           Emergence above ground              Leaves   <NA>
9              72                    First flower              Flowers   <NA>
10             73                First fruit ripe               Fruits   <NA>
11             75                       Full leaf               Leaves   <NA>
12             76                      First leaf               

R[write to console]: No phenophase provided for further validation.



Help on NULLType in module rpy2.rinterface_lib.sexp object:

class NULLType(Sexp)
 |  NULLType(*args, **kwargs)
 |  
 |  A singleton class for R's NULL.
 |  
 |  Method resolution order:
 |      NULLType
 |      Sexp
 |      rpy2.rinterface_lib._rinterface_capi.SupportsSEXP
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(self) -> bool
 |      This is always False.
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  rid
 |      ID of the underlying R object (memory address).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __sexp__
 |      Access to the underlying C pointer to the R object.
 |      
 |      When assigning a new SexpCapsule to this attribute, the
 |      R C-level type of the new capsule must be equal to t

#### Species and Phenophases

It's not possible to download all the species based on the "functional_type"! I get list of all the species and download all then concatenate them.

**Species id**: 
- DBF (1174,1446,...)
- EBF(120, 1861,...)
- GRA(1677,1032,1971,...)

**Phenophases**:

- breaking leaf buds (371,373)
- initial growth (482,492,508)

### Data preparation

In [14]:
# Load dataset
# years is needed! [start,end+1]!
# If there is no data for one year we get error---check years=[2013, 2014]

# A lot of queries returned no data Error! For example the one below

# dataset = NPNPhenor(species=1446, phenophase=373, years=[2007, 2018])
# dataset.download()
# df1 = dataset.load()
# df1

In [19]:
# In totall there are 1755 species!
print(len(ls_sp1))
print(len(ls_sp2))

338
137


In [20]:
#ls_species = ls_sp1
ls_species = ls_sp2
ls_phases = [371,373]
with tempfile.TemporaryDirectory() as tmpdirname:
    for i in ls_species:
        for j in ls_phases:
            try:
                dataset = NPNPhenor(species=i, phenophase=j, years=[2007, 2018])
                dataset.download()
                df1 = dataset.load()
                df1.to_csv(tmpdirname + "/species_" + str(i)+str(j)+ ".csv")
                print(df1.shape)
            except:
                pass

    # list all CSV files in working directory.
    os.chdir(tmpdirname)
    all_filenames = [i for i in glob.glob('*.csv')]
    print(all_filenames)
    
    # combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv( current_path + "/combined_csv.csv", index=False, encoding='utf-8-sig')

downloading /tmp/data/NPN/phenor_npn_data_103_371_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_103_373_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_104_371_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


...

R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_2139_373_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



[]


ValueError: No objects to concatenate

In [ ]:
# dataset = NPNPhenor(species=36, phenophase=483, years=[2012, 2013])
# dataset.download()
# df1 = dataset.load()
# list(df1.columns)